# Machine Learning Fairness Algorithms Evaluation

#### Fairness in Machine Learning
The aim of a fairness algorithm is to avoid the outcome decisions are being made unfairly to certain groups or individuals.
- Algorithms are made by humans and trained by data which may be biased
- However, there are many definitions of fairness that cannot be optimized at the same time


#### Learning fair representations (LFR)
The main idea: map each individual, represented as a data point in a given input space, to a probability distribution in a new representation space. The aim of this new representation is to lose any information that can identify whether the person belongs to the protected subgroup, while retaining as much other information as possible.

A discriminative clustering model, where the prototypes act as the clusters

#### Prejudice Remover Regularizer (PR)

ADD DETAILS


### Data Pre-processing

In [1]:
import csv
import pandas as pd

import numpy as np
from scipy.special import softmax
import scipy.optimize as optim
from sklearn.preprocessing import StandardScaler

In [2]:
data = 'https://raw.githubusercontent.com/juliamblake1/ADS-Project-4/main/data/compas-scores-two-years.csv'

In [3]:
compas_scores = pd.read_csv(data,header =0)
compas_scores.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
4,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,...,1,Low,2013-03-26,NaN,NaN,2,0,1102,0,0


In [4]:
compas_scores_race = compas_scores.loc[compas_scores['race'].isin(['Caucasian' ,'African-American'])]
compas_scores_race

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
6,8,edward riddle,edward,riddle,2014-02-19,Male,1974-07-23,41,25 - 45,Caucasian,...,2,Low,2014-02-19,2014-03-31,2014-04-18,14,5,40,1,1
8,10,elizabeth thieme,elizabeth,thieme,2014-03-16,Female,1976-06-03,39,25 - 45,Caucasian,...,1,Low,2014-03-16,2014-03-15,2014-03-18,0,2,747,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7207,10994,jarred payne,jarred,payne,2014-05-10,Male,1985-07-31,30,25 - 45,African-American,...,2,Low,2014-05-10,2015-10-22,2015-10-22,0,0,529,1,1
7208,10995,raheem smith,raheem,smith,2013-10-20,Male,1995-06-28,20,Less than 25,African-American,...,9,High,2013-10-20,2014-04-07,2014-04-27,0,0,169,0,0
7209,10996,steven butler,steven,butler,2013-11-23,Male,1992-07-17,23,Less than 25,African-American,...,5,Medium,2013-11-23,2013-11-22,2013-11-24,0,1,860,0,0
7210,10997,malcolm simmons,malcolm,simmons,2014-02-01,Male,1993-03-25,23,Less than 25,African-American,...,5,Medium,2014-02-01,2014-01-31,2014-02-02,0,1,790,0,0


In [5]:
# converting to binary data
df_one = pd.get_dummies(compas_scores_race["race"])
# print(df_one)
 
# display result
df_two = pd.concat((df_one, compas_scores_race), axis=1)
df_two = df_two.drop(["race"], axis=1)
df_two = df_two.drop(["Caucasian"], axis=1)
new_df = df_two.rename(columns={"African-American": "race"})
new_df

,race,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
1,1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,...,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,1,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,...,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
3,1,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,...,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
6,0,8,edward riddle,edward,riddle,2014-02-19,Male,1974-07-23,41,25 - 45,...,2,Low,2014-02-19,2014-03-31,2014-04-18,14,5,40,1,1
8,0,10,elizabeth thieme,elizabeth,thieme,2014-03-16,Female,1976-06-03,39,25 - 45,...,1,Low,2014-03-16,2014-03-15,2014-03-18,0,2,747,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7207,1,10994,jarred payne,jarred,payne,2014-05-10,Male,1985-07-31,30,25 - 45,...,2,Low,2014-05-10,2015-10-22,2015-10-22,0,0,529,1,1
7208,1,10995,raheem smith,raheem,smith,2013-10-20,Male,1995-06-28,20,Less than 25,...,9,High,2013-10-20,2014-04-07,2014-04-27,0,0,169,0,0
7209,1,10996,steven butler,steven,butler,2013-11-23,Male,1992-07-17,23,Less than 25,...,5,Medium,2013-11-23,2013-11-22,2013-11-24,0,1,860,0,0
7210,1,10997,malcolm simmons,malcolm,simmons,2014-02-01,Male,1993-03-25,23,Less than 25,...,5,Medium,2014-02-01,2014-01-31,2014-02-02,0,1,790,0,0


In [6]:
new_df.two_year_recid.value_counts(), new_df.race.value_counts()

(0    3283
 1    2867
 Name: two_year_recid, dtype: int64,
 1    3696
 0    2454
 Name: race, dtype: int64)

In [7]:
 new_df.v_decile_score.value_counts(), new_df.priors_count.value_counts(),  

(1     1501
 2      895
 3      854
 4      701
 5      602
 6      530
 7      413
 8      273
 9      266
 10     115
 Name: v_decile_score, dtype: int64,
 0     1710
 1     1166
 2      712
 3      504
 4      359
 5      299
 6      214
 7      191
 8      168
 9      136
 10     104
 11      91
 13      73
 12      70
 14      52
 15      49
 16      37
 17      34
 19      28
 18      25
 21      22
 22      20
 20      17
 23      15
 24      10
 25       8
 27       7
 28       7
 26       6
 29       5
 33       3
 30       2
 38       2
 36       1
 37       1
 35       1
 31       1
 Name: priors_count, dtype: int64)

## LFR Implementation
First the loss functions

In [8]:
np.random.seed(509)


def loss_x(x_new, x_initial):
    """
    Constrains the mapping to Z to be good description of X.
    Prototpyes should retain as much initial info as possible.

    difference is measured by squared sum of difference


    ARGS:
    x_new - Prototypes
    x_initial - raw data
    """
    return np.mean(np.sum(np.square((x_new - x_initial))))


def loss_y(y_true, y_predicted):
    """
    This loss term requires that the prediction of y is as accurate as possible:

    Computes log loss

    ARGS:
    y_true - (num_examples, )
    y_predicted - (num_examples, )
    """
    # logarithm is undefined in 0 which means y cant be 0 or 1 => we clip it
    y_true = np.clip(y_true, 1e-6, 0.999)
    y_predicted = np.clip(y_predicted, 1e-6, 0.999)

    log_loss = np.sum(y_true * np.log(y_predicted) +
                      (1. - y_true) * np.log(1. - y_predicted)) / len(y_true)

    return -log_loss


def loss_z(M_k_sensitive, M_k_non_sensitive):
    """
    Ensures statistical parity

    Calculates L1 distance

    Args:
    M_k_sensitive - (num_prototypes, )
    M_k_non_sensitive - (num_prototypes, )
    """
    return np.sum(np.abs(M_k_sensitive - M_k_non_sensitive))

In [9]:
def distances(X, v, alpha):
    """
    Calculates distance between initial data and each of the prototypes 
    Formula -> euclidean(x, v * alpha) (alpha is weight for each feature)

    ARGS:
    X - (num_examples, num_features)
    v - (num_prototypes, num_features)
    alpha - (num_features, 1)

    returns:
    dists - (num_examples, num_prototypes)
    """
    num_examples = X.shape[0]
    num_prototypes = v.shape[0]
    dists = np.zeros(shape=(num_examples, num_prototypes))

    # X = X.values  # converting to NumPy, this is needed in case you pass dataframe
    for i in range(num_examples):
        dist = np.square(X[i] - v)  # squarred distance
        dist_alpha = np.multiply(dist, alpha)  # multiplying by weights
        sum_ = np.sum(dist_alpha, axis=1)
        dists[i] = sum_

    return dists

In [10]:
def M_nk(dists):
    """
    define Mn,k as the probability that x maps to v

    Given the definitions of the prototypes as points in
    the input space, a set of prototypes induces a natural
    probabilistic mapping from X to Z via the softmax

    Since we already have distances calcutated we just map them to probabilities

    NOTE:
    minus distance because smaller the distance better the mapping

    ARGS:
    dists - (num_examples, num_prototypes)

    Return :
    mappings - (num_examples, num_prototypes)
    """
    return softmax(-dists, axis=1)  # specifying axis is important


def M_k(M_nk):
    """
    Calculate mean of the mapping for each prototype

    ARGS:
    M_nk - (num_examples, num_prototypes)

    Returns:
    M_k - mean of the mappings (num_prototypes, )
    """
    return np.mean(M_nk, axis=0)

In [11]:
def x_n_hat(M_nk, v):
    """
    Gets new representation of the data, 
    Performs simple dot product

    ARGS:
    M_nk - (num_examples, num_prototypes)
    v - (num_prototypes, num_features)

    Returns:
    x_n_hat - (num_examples, num_features)
    """
    return M_nk @ v


def y_hat(M_nk, w):
    """
    Function calculates labels in the new representation space
    Performs simple dot product

    ARGS:
    M_nk - (num_examples, num_prototypes)
    w - (num_prototypes, )

    returns:
    y_hat - (num_examples, )
    """
    return M_nk @ w

Now for the optimization

In [12]:
def optim_objective(params, data_sensitive, data_non_sensitive, y_sensitive,
                    y_non_sensitive,  inference=False, NUM_PROTOTYPES=10, A_x=0.01, A_y=0.1, A_z=0.5,
                    print_every=100):
    """
    Function gathers all the helper functions to calculate overall loss

    This is further passed to l-bfgs optimizer 

    ARGS:
    params - vector of length (2 * num_features + NUM_PROTOTYPES + NUM_PROTOTYPES * num_features)
    data_sensitive - instances belonging to senstive group (num_sensitive_examples, num_features)
    data_non_sensitive - similar to data_sensitive (num_non_senitive_examplesm num_features)
    y_sensitive - labels for sensitive group (num_sensitive_examples, )
    y_non_sensitive - similar to y_sensitive
    inference - (optional) if True than will return new dataset instead of loss
    NUM_PROTOTYPES - (optional), default 10
    A_x - (optional) hyperparameters for loss_X, default 0.01
    A_y - (optional) hyperparameters for loss_Y, default 1
    A_z - (optional) hyperparameters for loss_Z, default 0.5
    print_every - (optional) how often to print loss, default 100
    returns:
    if inference - False :
    float - A_x * L_x + A_y * L_y + A_z * L_z 
    if inference - True:
    x_hat_sensitive, x_hat_non_sensitive, y_hat_sensitive, y_hat_non_sensitive
    """
    optim_objective.iters += 1

    num_features = data_sensitive.shape[1]
    # extract values for each variable from params vector
    alpha_non_sensitive = params[:num_features]
    alpha_sensitive = params[num_features:2 * num_features]
    w = params[2 * num_features:2 * num_features + NUM_PROTOTYPES]
    v = params[2 * num_features + NUM_PROTOTYPES:].reshape(NUM_PROTOTYPES, num_features)

    dists_sensitive = distances(data_sensitive, v, alpha_sensitive)
    dists_non_sensitive = distances(data_non_sensitive, v, alpha_non_sensitive)

    # get probabilities of mappings
    M_nk_sensitive = M_nk(dists_sensitive)
    M_nk_non_sensitive = M_nk(dists_non_sensitive)

    # M_k only used for calcilating loss_y(statistical parity)
    M_k_sensitive = M_k(M_nk_sensitive)
    M_k_non_sensitive = M_k(M_nk_non_sensitive)
    L_z = loss_z(M_k_sensitive, M_k_non_sensitive)  # stat parity

    # get new representation of data
    x_hat_sensitive = x_n_hat(M_nk_sensitive, v)
    x_hat_non_sensitive = x_n_hat(M_nk_non_sensitive, v)
    # calculates how close new representation is to original data
    L_x_sensitive = loss_x(data_sensitive, x_hat_sensitive)
    L_x_non_sensitive = loss_x(data_non_sensitive, x_hat_non_sensitive)

    # get new values for labels
    y_hat_sensitive = y_hat(M_nk_sensitive, w)
    y_hat_non_sensitive = y_hat(M_nk_non_sensitive, w)
    # ensure how good new predictions are(log_loss)
    L_y_sensitive = loss_y(y_sensitive, y_hat_sensitive)
    L_y_non_sensitive = loss_y(y_non_sensitive, y_hat_non_sensitive)

    L_x = L_x_sensitive + L_x_non_sensitive
    L_y = L_y_sensitive + L_y_non_sensitive

    loss = A_x * L_x + A_y * L_y + A_z * L_z

    if optim_objective.iters % print_every == 0:
        print(f'loss on iteration {optim_objective.iters} : {loss}, L_x - {L_x * A_x} L_y - {L_y * A_y} L_z - {L_z * A_z}')
    
    if not inference:
        return loss
    if inference:
        return x_hat_sensitive, x_hat_non_sensitive, y_hat_sensitive, y_hat_non_sensitive

optim_objective.iters = 0

Now lets actually implement after we have all of the functions necessary

We are going to use a subset of the data for trial:

In [13]:
use_df = new_df[['two_year_recid', 'race', 'priors_count', 'v_decile_score', 'age']]
use_df

,two_year_recid,race,priors_count,v_decile_score,age
1,1,1,0,1,34
2,1,1,4,3,24
3,0,1,1,6,23
6,1,0,14,2,41
8,0,0,0,1,39
...,...,...,...,...,...
7207,1,1,0,2,30
7208,0,1,0,9,20
7209,0,1,0,5,23
7210,0,1,0,5,23


In [14]:
RACE_SENSITIVE = 0

# seperation into sensitive and non sensitive
data_sensitive = use_df[use_df.race > RACE_SENSITIVE]
data_non_sensitive = use_df[use_df.race <= RACE_SENSITIVE]
y_sensitive = data_sensitive.two_year_recid
y_non_sensitive = data_non_sensitive.two_year_recid

In [15]:
print (f'Dataset contains {use_df.shape[0]} examples and {use_df.shape[1]} features')
print (f'From which {data_sensitive.shape[0]} belong to sensitive group and {data_non_sensitive.shape[0]} to non sensitive group ')

Dataset contains 6150 examples and 5 features
From which 3696 belong to sensitive group and 2454 to non sensitive group 


In [16]:
del data_sensitive['two_year_recid']
del data_non_sensitive['two_year_recid']

# Standard Scaling
data_sensitive = StandardScaler().fit_transform(data_sensitive)
data_non_sensitive = StandardScaler().fit_transform(data_non_sensitive)


In [17]:
NUM_PROTOTYPES = 10
num_features = data_sensitive.shape[1]

params = np.random.uniform(size=(num_features * 2 + NUM_PROTOTYPES + NUM_PROTOTYPES * num_features))
# here we generate random weight for each of the features both for sensitive data
# and for non sensitive, hence num_features*2(in paper this is denoted as alpha)
# alphas are used for calculating distances


In [18]:
# Then NUM_PROTOTYPES is a weight for each prototype, this is multiplied with 
# M_nk s and used for calculating y_hat

# Next is NUM_PROTOTYPES * num_features which is v(in paper), this is also used
# for calculating distances


bnd = [] # This is needed for l-bfgs algorithm
for i, _ in enumerate(params):
    if i < num_features * 2 or i >= num_features * 2 + NUM_PROTOTYPES:
        bnd.append((None, None))
    else:
        bnd.append((0, 1))

In [19]:
new_params = optim.fmin_l_bfgs_b(optim_objective, x0=params, epsilon=1e-5,
                                  args=(data_sensitive, data_non_sensitive,
                                        y_sensitive, y_non_sensitive),
                                  bounds=bnd, approx_grad=True, maxfun=1000,
                                  maxiter=1000)[0]


x_hat_senitive, x_hat_nons, y_hat_sens, y_hat_nons = optim_objective(new_params,data_sensitive, data_non_sensitive,
                                        y_sensitive, y_non_sensitive, inference=False)


loss on iteration 100 : 151.85569488772356, L_x - 151.65946108403293 L_y - 0.14425030879802944 L_z - 0.0519834948925982
loss on iteration 200 : 39.96647896619465, L_x - 39.691357292317036 L_y - 0.17105209786542533 L_z - 0.10406957601218769
loss on iteration 300 : 26.763019529673617, L_x - 26.498199308874383 L_y - 0.1672788151737492 L_z - 0.09754140562548326
loss on iteration 400 : 20.43543421761302, L_x - 20.197125576976127 L_y - 0.15410188024860427 L_z - 0.08420676038829
loss on iteration 500 : 17.936021821123116, L_x - 17.70707425010495 L_y - 0.1504740843096172 L_z - 0.07847348670854905
loss on iteration 600 : 15.418761747154589, L_x - 15.195824332148804 L_y - 0.14748900431566594 L_z - 0.07544841069011887
loss on iteration 700 : 100.4255804349174, L_x - 100.02765991623502 L_y - 0.14011248417582284 L_z - 0.25780803450657186
loss on iteration 800 : 11.930876728999314, L_x - 11.748865634385734 L_y - 0.1418637307554593 L_z - 0.04014736385812077
loss on iteration 900 : 11.088643160071914,

TypeError: cannot unpack non-iterable numpy.float64 object

In [20]:
print ('Done')

Done
